<a href="https://colab.research.google.com/github/LYULU2/FakeNewsDetection/blob/main/TopicModelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

from google.colab import drive
drive.mount("/content/drive")

Found GPU at: /device:GPU:0
Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import string
from gensim import matutils, models
import scipy.sparse
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
def process_text(text):
    combined = ' '.join(text)
    return combined

In [ ]:
import re
import nltk
nltk.download('stopwords')
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

def clean_text(text):
  stemmed_text = []
  porter_stemmer = PorterStemmer()
  text = re.sub('[^a-zA-Z]', ' ', text)
  text = re.sub(r'http\S+', '', text)
  text = re.sub(r'www\.[a-z]+\.at', '', text)
  result = re.sub('[%s]' % re.escape(string.punctuation), '', text)
  result = result.lower()
  result = result.split()
  result = [r for r in result if r not in set(stopwords.words('english'))]
  stemmed_result = [porter_stemmer.stem(r) for r in result]
  stemmed_text.append(" ".join(stemmed_result))
  return stemmed_text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
data_alt_before = pd.read_csv('/content/drive/MyDrive/Alt-right-2018-01-01--2020-02-29.csv', delimiter=',')
data_anti_before = pd.read_csv('/content/drive/MyDrive/AVaxxer-2018-01-01--2020-02-29.csv', delimiter=',')
data_alt_after = pd.read_csv('/content/drive/MyDrive/Alt-right-2020-03-01--2021-09-19.csv', delimiter=',')
data_anti_after = pd.read_csv('/content/drive/MyDrive/AVaxxer-2020-03-01--2021-09-19.csv', delimiter=',')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (37,38,39) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (39) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
data_alt_before.head()

,Page Name,User Name,Facebook Id,Page Category,Page Admin Top Country,Page Description,Page Created,Likes at Posting,Followers at Posting,Post Created,Post Created Date,Post Created Time,Type,Total Interactions,Likes,Comments,Shares,Love,Wow,Haha,Sad,Angry,Care,Video Share Status,Is Video Owner?,Post Views,Total Views,Total Views For All Crossposts,Video Length,URL,Message,Link,Final Link,Image Text,Link Text,Description,Sponsor Id,Sponsor Name,Sponsor Category,Overperforming Score (weighted — Likes 1x Shares 1x Comments 1x Love 1x Wow 1x Haha 1x Sad 1x Angry 1x Care 1x )
0,The Conservative,the.conservative2,605636446121211,MEDIA_NEWS_COMPANY,US,"""They who can give up essential liberty to obt...",2013-06-05 21:30:50,209150.0,192427.0,2020-02-28 19:02:03 PST,2020-02-28,19:02:03,Link,18,5,5,2,2,0,3,0,1,0,NaN,-,0,0,0,NaN,https://www.facebook.com/605636446121211/posts...,NaN,http://dlvr.it/RQyPhP,https://www.dailywire.com/news/aoc-rips-pence-...,NaN,AOC Rips Pence As ‘Anti-Science.’ Cruz Shreds ...,When President Donald Trump announced Wednesda...,NaN,NaN,NaN,-2.67
1,Republican Security Council,RSC.GOP,101173139981866,POLITICAL_PARTY,US,We are part of the GOP's national security win...,2011-07-12 21:47:03,105681.0,105770.0,2020-02-28 18:59:24 PST,2020-02-28,18:59:24,Photo,"3,366",1531,115,1138,479,99,1,3,0,0,NaN,-,0,0,0,NaN,https://www.facebook.com/101173139981866/posts...,Happy Birthday To Madeleine Carroll. She Was T...,https://www.facebook.com/RSC.GOP/photos/a.1095...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.07
2,Texas for Donald Trump 2020,Texasfordonaldtrump2020,869110696505336,FAN_PAGE,US,Disclaimer: This page is owned by a private US...,2015-08-18 21:02:41,296505.0,295158.0,2020-02-28 17:40:04 PST,2020-02-28,17:40:04,Link,"3,378",595,148,2011,9,154,5,39,417,0,NaN,-,0,0,0,NaN,https://www.facebook.com/869110696505336/posts...,NaN,https://pjmedia.com/trending/fact-check-obama-...,NaN,NaN,Fact-Check: Obama Waited Until 'Millions' Infe...,"""Let's call it Trumpvirus,"" urged a New York T...",NaN,NaN,NaN,20.98
3,United Deplorables Of America,UnitedDeplorablesOfAmerica,427436237618707,ACTIVITY_GENERAL,US,We Are Trump's Deplorables and Together We Wil...,2017-04-12 04:47:02,328707.0,334862.0,2020-02-28 16:59:45 PST,2020-02-28,16:59:45,Link,"1,234",226,281,227,4,32,131,33,300,0,NaN,-,0,0,0,NaN,https://www.facebook.com/427436237618707/posts...,Pelosi Has Absolutely Lost Her Mind! 😲😒,https://breakingfirst.com/nancy-pelosis-sanity...,NaN,NaN,Nancy Pelosi’s Sanity & Integrity Questioned A...,Democratic House Speaker Nancy Pelosi can’t se...,NaN,NaN,NaN,1.54
4,Colorado for Donald Trump 2020,Colorado4DonaldTrump,1619236631661479,FAN_PAGE,US,Disclaimer: This page is owned by a private US...,2015-08-18 03:21:10,162575.0,159465.0,2020-02-28 06:00:59 PST,2020-02-28,06:00:59,Native Video,20,19,0,1,0,0,0,0,0,0,share,No,671,619297,0,00:09:12,https://www.facebook.com/1619236631661479/post...,NaN,https://www.facebook.com/FoxNews/videos/520705...,NaN,NaN,Fox News,“We have an opportunity to do exactly what the...,NaN,NaN,NaN,-4.25


Pre-processing

In [ ]:
def pre_process(data):
  data['Message'] = data['Message'].fillna('')
  text = data['Message'].astype('string').tolist()
  text_combined = process_text(text)
  text_cleaned = clean_text(text_combined)
  return text_cleaned

alt_before = pre_process(data_alt_before)
alt_after = pre_process(data_alt_after)
anti_before = pre_process(data_anti_before)
anti_after = pre_process(data_anti_after)

combined_dict = {"alt-before":alt_before, "anti-before": anti_before,"alt-after":alt_after,"anti-after":anti_after}
data_df = pd.DataFrame.from_dict(combined_dict, orient='index', columns=['text'])
data_df

,text
alt-before,happi birthday madelein carrol highest paid ac...
anti-before,realli gonna kill everybodi dmt spiritu scienc...
alt-after,classifi testimoni opposit biden surrend under...
anti-after,see break fda panel overwhelmingli reject pfiz...


In [ ]:
cv = CountVectorizer(stop_words='english')
#cv = CountVectorizer()
data_cv = cv.fit_transform(data_df.text)
data_dtm = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names())
data_dtm.index = data_df.index
data_dtm

,aa,aaa,aaaaaaaab,aaaaaaand,aaaaand,aaaac,aaaand,aaaci,aaacnlkw,aaadb,aaadnnb,aaaexzhzq,aaagl,aaah,aaakxza,aaamp,aaand,aaap,aaargu,aab,aabg,aabi,aabot,aabydc,aac,aacd,aaceihti,aach,aacijourn,aaciw,aacjo,aacp,aacpl,aacr,aacspjemqhlt,aad,aadaenpt,aadb,aadewk,aadf,...,zzikieru,zzit,zzjv,zzjye,zzkinr,zzkp,zzkusmlj,zzl,zzm,zzmale,zzmit,zzn,zzo,zzon,zzp,zzpb,zzqh,zzqxi,zzr,zzrom,zzsn,zzsqif,zzsvhqga,zztiyhtu,zztn,zztopmloweyfjgiyi,zzu,zzuk,zzuusn,zzvhpw,zzvptbxqh,zzw,zzwqlk,zzwyiek,zzxaepj,zzxtcm,zzxtftzr,zzybvuyyao,zzzivafq,zzzvlmcp
alt-before,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
anti-before,104,23,1,1,0,2,1,2,1,0,2,0,2,1,1,0,2,1,0,3,1,45,3,1,6,1,0,1,2,0,2,1,0,5,1,3,1,0,1,1,...,2,0,0,1,0,1,0,1,0,0,0,0,1,0,2,0,0,2,1,2,2,1,0,1,2,0,1,0,2,0,0,0,1,1,1,6,1,1,1,0
alt-after,0,21,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
anti-after,87,10,0,0,1,0,0,0,0,1,0,1,0,1,0,1,0,0,1,8,0,0,2,0,12,0,2,0,0,1,0,0,1,0,0,3,0,2,0,0,...,0,1,2,0,1,0,1,0,1,2,2,1,0,2,0,1,1,0,0,0,0,0,2,0,0,1,0,1,0,1,1,1,0,0,0,0,0,0,0,1


In [ ]:
tdm = data_dtm.transpose()
tdm.head()

,alt-before,anti-before,alt-after,anti-after
aa,1,104,0,87
aaa,0,23,21,10
aaaaaaaab,0,1,0,0
aaaaaaand,0,1,0,0
aaaaand,0,0,0,1


In [ ]:
sparse_counts = scipy.sparse.csr_matrix(tdm)
corpus = matutils.Sparse2Corpus(sparse_counts)

id2word = dict((v, k) for k, v in cv.vocabulary_.items())

In [ ]:
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=2, passes=20)
for idx, topic in lda.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.016*"covid" + 0.014*"vaccin" + 0.013*"la" + 0.009*"en" + 0.009*"el" + 0.008*"que" + 0.005*"lo" + 0.004*"peopl" + 0.004*"se" + 0.004*"health"
Topic: 1 
Words: 0.021*"vaccin" + 0.005*"health" + 0.004*"peopl" + 0.004*"children" + 0.004*"state" + 0.003*"time" + 0.003*"year" + 0.003*"truth" + 0.003*"govern" + 0.003*"said"


In [ ]:
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=3, passes=20)
for idx, topic in lda.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.023*"vaccin" + 0.005*"health" + 0.004*"children" + 0.004*"peopl" + 0.004*"state" + 0.004*"truth" + 0.003*"time" + 0.003*"year" + 0.003*"govern" + 0.003*"freedom"
Topic: 1 
Words: 0.016*"covid" + 0.013*"vaccin" + 0.013*"la" + 0.008*"en" + 0.008*"el" + 0.008*"que" + 0.005*"lo" + 0.004*"peopl" + 0.004*"se" + 0.004*"health"
Topic: 2 
Words: 0.001*"vaccin" + 0.001*"covid" + 0.001*"el" + 0.001*"la" + 0.001*"en" + 0.000*"que" + 0.000*"lo" + 0.000*"peopl" + 0.000*"health" + 0.000*"por"


In [ ]:
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=4, passes=20)
for idx, topic in lda.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.028*"vaccin" + 0.005*"health" + 0.005*"children" + 0.004*"peopl" + 0.004*"time" + 0.004*"state" + 0.003*"truth" + 0.003*"year" + 0.003*"parent" + 0.003*"medic"
Topic: 1 
Words: 0.017*"covid" + 0.014*"vaccin" + 0.014*"la" + 0.009*"en" + 0.009*"el" + 0.008*"que" + 0.005*"lo" + 0.004*"peopl" + 0.004*"se" + 0.004*"health"
Topic: 2 
Words: 0.010*"podcast" + 0.006*"presid" + 0.006*"said" + 0.005*"conserv" + 0.005*"daili" + 0.005*"govern" + 0.005*"zambia" + 0.005*"peopl" + 0.004*"covid" + 0.004*"trump"
Topic: 3 
Words: 0.001*"vaccin" + 0.001*"covid" + 0.000*"en" + 0.000*"la" + 0.000*"el" + 0.000*"peopl" + 0.000*"que" + 0.000*"lo" + 0.000*"se" + 0.000*"new"


In [ ]:
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=5, passes=20)
for idx, topic in lda.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.028*"vaccin" + 0.005*"health" + 0.005*"children" + 0.004*"peopl" + 0.004*"time" + 0.004*"state" + 0.003*"truth" + 0.003*"year" + 0.003*"parent" + 0.003*"medic"
Topic: 1 
Words: 0.017*"covid" + 0.014*"vaccin" + 0.014*"la" + 0.009*"en" + 0.009*"el" + 0.008*"que" + 0.005*"lo" + 0.004*"peopl" + 0.004*"se" + 0.004*"health"
Topic: 2 
Words: 0.001*"vaccin" + 0.000*"covid" + 0.000*"la" + 0.000*"peopl" + 0.000*"que" + 0.000*"health" + 0.000*"el" + 0.000*"time" + 0.000*"children" + 0.000*"truth"
Topic: 3 
Words: 0.010*"podcast" + 0.006*"presid" + 0.006*"said" + 0.005*"conserv" + 0.005*"daili" + 0.005*"govern" + 0.005*"zambia" + 0.005*"peopl" + 0.004*"covid" + 0.004*"trump"
Topic: 4 
Words: 0.001*"vaccin" + 0.001*"covid" + 0.001*"la" + 0.001*"el" + 0.001*"que" + 0.000*"en" + 0.000*"se" + 0.000*"lo" + 0.000*"del" + 0.000*"peopl"


In [ ]:
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=2, passes=80)
for idx, topic in lda.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.021*"vaccin" + 0.005*"health" + 0.004*"peopl" + 0.004*"children" + 0.004*"state" + 0.003*"time" + 0.003*"year" + 0.003*"truth" + 0.003*"govern" + 0.003*"said"
Topic: 1 
Words: 0.016*"covid" + 0.014*"vaccin" + 0.013*"la" + 0.009*"en" + 0.009*"el" + 0.008*"que" + 0.005*"lo" + 0.004*"peopl" + 0.004*"se" + 0.004*"health"


In [ ]:
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=3, passes=80)
for idx, topic in lda.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.016*"covid" + 0.014*"vaccin" + 0.014*"la" + 0.009*"en" + 0.009*"el" + 0.008*"que" + 0.005*"lo" + 0.004*"peopl" + 0.004*"se" + 0.004*"health"
Topic: 1 
Words: 0.010*"podcast" + 0.006*"presid" + 0.005*"said" + 0.005*"conserv" + 0.005*"daili" + 0.005*"govern" + 0.005*"zambia" + 0.004*"peopl" + 0.004*"covid" + 0.004*"trump"
Topic: 2 
Words: 0.027*"vaccin" + 0.005*"health" + 0.005*"children" + 0.004*"peopl" + 0.004*"time" + 0.004*"state" + 0.003*"truth" + 0.003*"year" + 0.003*"parent" + 0.003*"medic"


In [ ]:
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=4, passes=80)
for idx, topic in lda.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.000*"vaccin" + 0.000*"la" + 0.000*"covid" + 0.000*"en" + 0.000*"peopl" + 0.000*"health" + 0.000*"time" + 0.000*"el" + 0.000*"govern" + 0.000*"news"
Topic: 1 
Words: 0.028*"vaccin" + 0.005*"health" + 0.005*"children" + 0.004*"peopl" + 0.004*"time" + 0.004*"state" + 0.003*"truth" + 0.003*"year" + 0.003*"parent" + 0.003*"medic"
Topic: 2 
Words: 0.010*"podcast" + 0.006*"presid" + 0.006*"said" + 0.005*"conserv" + 0.005*"daili" + 0.005*"govern" + 0.005*"zambia" + 0.005*"peopl" + 0.004*"covid" + 0.004*"trump"
Topic: 3 
Words: 0.017*"covid" + 0.014*"vaccin" + 0.014*"la" + 0.009*"en" + 0.009*"el" + 0.008*"que" + 0.005*"lo" + 0.004*"peopl" + 0.004*"se" + 0.004*"health"


In [ ]:
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=5, passes=80)
for idx, topic in lda.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.010*"podcast" + 0.006*"presid" + 0.006*"said" + 0.005*"conserv" + 0.005*"daili" + 0.005*"govern" + 0.005*"zambia" + 0.005*"peopl" + 0.004*"covid" + 0.004*"trump"
Topic: 1 
Words: 0.000*"vaccin" + 0.000*"peopl" + 0.000*"covid" + 0.000*"govern" + 0.000*"la" + 0.000*"health" + 0.000*"truth" + 0.000*"said" + 0.000*"time" + 0.000*"year"
Topic: 2 
Words: 0.000*"vaccin" + 0.000*"covid" + 0.000*"la" + 0.000*"que" + 0.000*"en" + 0.000*"el" + 0.000*"peopl" + 0.000*"lo" + 0.000*"health" + 0.000*"se"
Topic: 3 
Words: 0.028*"vaccin" + 0.005*"health" + 0.005*"children" + 0.004*"peopl" + 0.004*"time" + 0.004*"state" + 0.003*"truth" + 0.003*"year" + 0.003*"parent" + 0.003*"medic"
Topic: 4 
Words: 0.017*"covid" + 0.014*"vaccin" + 0.014*"la" + 0.009*"en" + 0.009*"el" + 0.008*"que" + 0.005*"lo" + 0.004*"peopl" + 0.004*"se" + 0.004*"health"


In [ ]:
import pickle
outfile = open("cleaned_text",'wb')
pickle.dump(data_df,outfile)
outfile.close()

In [ ]:
for t in corpus_transformed:
  print(t)

[(0, 0.20086189), (1, 0.7991381)]
[(1, 0.9996052)]
[(0, 0.9497707), (1, 0.050229285)]
[(0, 0.99401087)]


In [ ]:
ct = []
for t in corpus_transformed:
  for (a,b) in t:
    if b>0.7:
      ct +=[(a,b)]

Take Nouns

In [ ]:
from nltk import word_tokenize, pos_tag
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

def nouns(text):
    '''Given a string of text, tokenize the text and pull out only the nouns.'''
    is_noun = lambda pos: pos[:2] == 'NN'
    tokenized = word_tokenize(text)
    all_nouns = [word for (word, pos) in pos_tag(tokenized) if is_noun(pos)] 
    return ' '.join(all_nouns)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:
data_clean = pd.read_pickle('cleaned_text')
data_clean

,text
alt-before,happi birthday madelein carrol highest paid ac...
anti-before,realli gonna kill everybodi dmt spiritu scienc...
alt-after,classifi testimoni opposit biden surrend under...
anti-after,see break fda panel overwhelmingli reject pfiz...


In [ ]:
data_nouns = data_clean.text.apply(nouns)

alt-before     happi birthday actress world ww receiv preside...
anti-before    realli gon dmt spiritu scienc travers feat min...
alt-after      classifi testimoni opposit surrend underscor m...
anti-after     panel overwhelmingli pfizer booster issu boost...
Name: text, dtype: object

In [ ]:
data_nouns = pd.DataFrame(data_nouns)
data_nouns

,text
alt-before,happi birthday actress world ww receiv preside...
anti-before,realli gon dmt spiritu scienc travers feat min...
alt-after,classifi testimoni opposit surrend underscor m...
anti-after,panel overwhelmingli pfizer booster issu boost...


In [ ]:
from sklearn.feature_extraction import text
add_stop_words = ['el', 'la', 'news', 'year', 'vaccine', 'covid', 'people']
stop_words = text.ENGLISH_STOP_WORDS.union(add_stop_words)

cv = CountVectorizer(stop_words=stop_words)
data_cv = cv.fit_transform(data_nouns.text)
data_dtm = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names())
data_dtm.index = data_nouns.index
tdm = data_dtm.transpose()
sparse_counts = scipy.sparse.csr_matrix(tdm)
corpus = matutils.Sparse2Corpus(sparse_counts)

id2word = dict((v, k) for k, v in cv.vocabulary_.items())

In [ ]:
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=3, passes=20)
for idx, topic in lda.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.032*"vaccin" + 0.008*"health" + 0.008*"children" + 0.006*"time" + 0.005*"state" + 0.005*"peopl" + 0.005*"parent" + 0.005*"truth" + 0.004*"freedom" + 0.004*"day"
Topic: 1 
Words: 0.012*"podcast" + 0.008*"presid" + 0.008*"daili" + 0.008*"conserv" + 0.006*"zambia" + 0.006*"state" + 0.006*"nation" + 0.005*"peopl" + 0.005*"trump" + 0.005*"time"
Topic: 2 
Words: 0.017*"vaccin" + 0.011*"que" + 0.006*"health" + 0.006*"peopl" + 0.004*"lo" + 0.004*"por" + 0.004*"state" + 0.004*"time" + 0.004*"para" + 0.004*"coronaviru"


In [ ]:
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=4, passes=20)
for idx, topic in lda.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.005*"vaccin" + 0.002*"health" + 0.002*"children" + 0.001*"state" + 0.001*"truth" + 0.001*"time" + 0.001*"peopl" + 0.001*"doctor" + 0.001*"day" + 0.001*"freedom"
Topic: 1 
Words: 0.013*"podcast" + 0.009*"presid" + 0.008*"daili" + 0.008*"conserv" + 0.006*"zambia" + 0.006*"state" + 0.006*"nation" + 0.005*"peopl" + 0.005*"trump" + 0.005*"time"
Topic: 2 
Words: 0.032*"vaccin" + 0.008*"health" + 0.008*"children" + 0.006*"time" + 0.006*"state" + 0.005*"peopl" + 0.005*"parent" + 0.005*"truth" + 0.004*"freedom" + 0.004*"day"
Topic: 3 
Words: 0.017*"vaccin" + 0.011*"que" + 0.006*"health" + 0.006*"peopl" + 0.004*"lo" + 0.004*"por" + 0.004*"state" + 0.004*"time" + 0.004*"para" + 0.004*"coronaviru"


In [ ]:
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=4, passes=80)
for idx, topic in lda.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.017*"vaccin" + 0.011*"que" + 0.006*"health" + 0.006*"peopl" + 0.004*"lo" + 0.004*"por" + 0.004*"state" + 0.004*"time" + 0.004*"para" + 0.004*"coronaviru"
Topic: 1 
Words: 0.013*"podcast" + 0.009*"presid" + 0.008*"daili" + 0.008*"conserv" + 0.006*"zambia" + 0.006*"state" + 0.006*"nation" + 0.005*"peopl" + 0.005*"trump" + 0.005*"time"
Topic: 2 
Words: 0.032*"vaccin" + 0.008*"health" + 0.008*"children" + 0.006*"time" + 0.006*"state" + 0.005*"peopl" + 0.005*"parent" + 0.005*"truth" + 0.004*"freedom" + 0.004*"day"
Topic: 3 
Words: 0.000*"vaccin" + 0.000*"peopl" + 0.000*"que" + 0.000*"health" + 0.000*"state" + 0.000*"children" + 0.000*"case" + 0.000*"report" + 0.000*"time" + 0.000*"world"


topic 0: 
topic 1: political
topic 2: free will

In [ ]:
corpus_transformed = lda[corpus]
for t in corpus_transformed:
  print(t)

[(1, 0.9699018), (2, 0.030094396)]
[(2, 0.9997456)]
[(1, 0.9997837)]
[(0, 0.9946027)]


In [ ]:
ct = []
for t in corpus_transformed:
  for (a,b) in t:
    if b>0.7:
      ct +=[(a,b)]

In [ ]:
list(zip([a for (a,b) in ct], data_dtm.index))

[(1, 'alt-before'), (2, 'anti-before'), (1, 'alt-after'), (0, 'anti-after')]